In [1]:
from loader import tanaka_corpus
from prep import make_data_set

In [2]:
train_sentences = tanaka_corpus(is_train=True)
test_sentences = tanaka_corpus(is_train=False)

In [3]:
data_set = make_data_set(*train_sentences, *test_sentences)

In [19]:
!poetry run python -m spacy download en
!poetry run python -m spacy download de
!poetry run python -m spacy download ja_core_news_md

You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/ryutayamamoto/.ghq/github.com/Ryuta-Yamamoto/ML-study/.venv/lib/python3.7/site-packages/en_core_web_sm
-->
/Users/ryutayamamoto/.ghq/github.com/Ryuta-Yamamoto/ML-study/.venv/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/Users/ryutayamamoto/.ghq/github.com/Ryuta-Yamamoto/ML-study/.venv/lib/python3.7/site-packages/de_core_news_sm
-->
/Users/ryutayamamoto/.ghq/github.com/Ryuta-Yamamoto/ML-study/.venv/lib/python3.7/site-packages/spacy/data/de
You can now load the model via spacy.load('de')
     |████████████████████████████████| 38.8MB 582kB/s eta 0:00:0105     |███████▏                        | 8.6MB 866kB/s eta 0:00:35     |████████▌                       | 10.3MB 845kB/s eta 0:00:34     |███████████████▉                | 19.3MB 1.8MB/s eta 0:00:12�███████████████                | 19.5MB 1.8MB/s eta 0:00:12�███████████████▏               | 19.6MB 1.8MB/s eta 0:00:11     |████████████████████▉           | 25.2MB 610kB/s eta 0:00:23 eta 0:00:32kB/s eta 0:00:32
     |████████████████████████████████| 296kB 3.8MB/

     |████████████████████████████████| 1.9MB 366kB/s eta 0:00:01
  Created wheel for ja-core-news-md: filename=ja_core_news_md-2.3.1-cp37-none-any.whl size=39028868 sha256=8a638a19a53eb0b5d62f3962690f8a56cfd239a1d1be67fea8c3cf1a4a79e89d
  Stored in directory: /private/var/folders/_0/f9sp8z895399jqc5ryb95d3m0000gn/T/pip-ephem-wheel-cache-_o4qxp2u/wheels/5e/3c/e2/f15e974be2d7ab2420d666571207fa8e18cebc1ce7de3201cb
  Created wheel for sudachidict-core: filename=SudachiDict_core-20200330-cp37-none-any.whl size=71080894 sha256=96dc111dc5e57c8679f1704e7afed01928acdd577b960e22aa83ebf7e2223621
  Stored in directory: /private/var/folders/_0/f9sp8z895399jqc5ryb95d3m0000gn/T/pip-ephem-wheel-cache-_o4qxp2u/wheels/fc/69/08/4cbe227806f9a433907d311020e1cb303484e4a0f3b56471b7
Successfully built ja-core-news-md sudachidict-core
You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('ja_core_news_md')


In [53]:
from torchtext.datasets import Multi30k, TranslationDataset
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize = "spacy",
            tokenizer_language="en",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

TRG = Field(tokenize = "spacy",
            tokenizer_language="ja_core_news_md",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

train_data = TranslationDataset(
    path="data/tanaka_corpus/small_parallel_enja/dev.",
    exts=("en", "ja"),
    fields = (SRC, TRG)
)

In [54]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [55]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator,  = BucketIterator.splits(
    (train_data,),
    batch_size = BATCH_SIZE,
    device = device)

In [31]:
import random
import time
from typing import Tuple, Optional
import numpy as np

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

256

In [195]:
# src_vcab = 1000
# trg_vcab = 1000

class MyTransformer(nn.Module):
    def __init__(
        self, 
        src_emb: nn.Module, 
        trg_emb: nn.Module,
        transformer: nn.Module,
        pad_index: int,
        device: Optional[torch.device] = None
    ):
        super().__init__()
        self.src_emb = src_emb
        self.trg_emb = trg_emb
        self.transformer = transformer
        self.pad_index = pad_index
        self.out = nn.Linear(
            in_features=transformer.d_model,
            out_features=trg_emb.num_embeddings
        )
        if not device:
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(device)
        
    def forward(self, src: Tensor, trg: Tensor):
        src_tensor = self.src_emb(src)
        trg_tensor = self.trg_emb(trg)
        trg_mask = torch.triu(torch.ones(len(trg_tensor), len(trg_tensor))).T
        pad_mask = (src == self.pad_index).T
        hidden = self.transformer(
            src_tensor, 
            trg_tensor, 
            tgt_mask=trg_mask, 
            src_key_padding_mask=pad_mask
        )
        return self.out(hidden)
        

def make_transformer(
        src_vcab: int,
        trg_vcab: int,
        embedding_dim: int,
        n_head: int = 16,
        num_encoder_layers: int = 3
):
    src_emb = nn.Embedding(
        num_embeddings=src_vcab, 
        embedding_dim=embedding_dim
    )
    trg_emb = nn.Embedding(
        num_embeddings=trg_vcab,
        embedding_dim=embedding_dim
    )
    transformer = nn.Transformer(
        d_model=embedding_dim,
        nhead=n_head, 
        num_encoder_layers=num_encoder_layers
    )
    return MyTransformer(
        src_emb=src_emb,
        trg_emb=trg_emb,
        transformer=transformer,
        pad_index=SRC.vocab.stoi[SRC.pad_token]
    )
        

In [196]:
model = make_transformer(
    src_vcab=len(SRC.vocab.itos),
    trg_vcab=len(TRG.vocab.itos),
    embedding_dim=256,
)

In [197]:
optimizer = optim.Adam(model.parameters())

In [198]:
import math
import time


def train(model: nn.Module,
          iterator: BucketIterator,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, batch in enumerate(iterator):

        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()
        
        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def epoch_time(
        start_time: int,
        end_time: int
):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [211]:
def estimate(
        model: nn.Module,
        iterator: BucketIterator
):
    model.eval()
    
    with torch.no_grad():
        for _, batch in enumerate(iterator):
            src = batch.src
            
            sos = torch.tensor([[TRG.vocab.stoi[TRG.init_token]] * src.shape[1]])
            trg = sos
            
            while not is_end(trg):
                output = model(src, trg)
#                 trg = torch.cat([trg, choice(output)])
                arr = output.argmax(axis=-1).reshape((output.shape[0], -1))
                trg = torch.cat([sos, arr])
    return trg


def choice(tensor):
    return tensor[-1].argmax(axis=-1).reshape((1, -1))

def is_end(tensor):
    # 20単語以上で終了
    return tensor.shape[0] >= 20

def to_text(seqs):
    return [' '.join(arr) for arr in np.array(TRG.vocab.itos)[seqs].T]

In [200]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

PAD_IDX = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
#     valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')

# test_loss = evaluate(model, test_iterator, criterion)

# print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

Epoch: 01 | Time: 0m 20s
	Train Loss: 5.206 | Train PPL: 182.290
Epoch: 02 | Time: 0m 15s
	Train Loss: 4.466 | Train PPL:  87.019
Epoch: 03 | Time: 0m 24s
	Train Loss: 4.331 | Train PPL:  76.039
Epoch: 04 | Time: 0m 18s
	Train Loss: 4.230 | Train PPL:  68.703
Epoch: 05 | Time: 0m 17s
	Train Loss: 4.300 | Train PPL:  73.668
Epoch: 06 | Time: 0m 22s
	Train Loss: 3.712 | Train PPL:  40.937
Epoch: 07 | Time: 0m 21s
	Train Loss: 3.344 | Train PPL:  28.337
Epoch: 08 | Time: 0m 26s
	Train Loss: 2.917 | Train PPL:  18.479
Epoch: 09 | Time: 0m 20s
	Train Loss: 2.379 | Train PPL:  10.791
Epoch: 10 | Time: 0m 22s
	Train Loss: 1.953 | Train PPL:   7.049


In [114]:
torch.cat([o, o]).shape

torch.Size([36, 128, 380])

In [121]:
trg = torch.tensor([[TRG.vocab.stoi[TRG.init_token]] * len(src)])

NameError: name 'src' is not defined

In [118]:
o[0, 0].reshape((1, -1)).shape

torch.Size([1, 380])

In [166]:
torch.triu(torch.ones((2, 2))).T

tensor([[1., 0.],
        [1., 1.]])

tensor([[8, 8, 8,  ..., 7, 8, 7],
        [3, 0, 3,  ..., 7, 0, 3],
        [0, 0, 0,  ..., 0, 0, 3],
        ...,
        [8, 3, 3,  ..., 3, 3, 0],
        [0, 3, 3,  ..., 3, 3, 3],
        [3, 3, 3,  ..., 3, 3, 3]])

In [155]:
o[:, 1, :]

tensor([[-1.1723, -0.9570, -1.3386,  ..., -0.0545, -0.2099, -0.4432],
        [ 0.4067, -1.1410, -1.3601,  ...,  0.2576, -0.5417, -0.4260],
        [-1.9829, -2.1971, -1.9372,  ..., -1.6308, -0.5584, -1.1724],
        ...,
        [-1.9829, -2.1971, -1.9372,  ..., -1.6308, -0.5584, -1.1724],
        [-1.9829, -2.1971, -1.9372,  ..., -1.6308, -0.5584, -1.1724],
        [-1.9829, -2.1971, -1.9372,  ..., -1.6308, -0.5584, -1.1724]])

In [154]:
o[:, 0, :]

tensor([[-1.1728, -0.9571, -1.3386,  ..., -0.0545, -0.2097, -0.4432],
        [ 0.4062, -1.1411, -1.3600,  ...,  0.2576, -0.5414, -0.4259],
        [-1.9829, -2.1972, -1.9371,  ..., -1.6308, -0.5582, -1.1725],
        ...,
        [-1.9829, -2.1972, -1.9371,  ..., -1.6308, -0.5582, -1.1725],
        [-1.9829, -2.1972, -1.9371,  ..., -1.6308, -0.5582, -1.1725],
        [-1.9829, -2.1972, -1.9371,  ..., -1.6308, -0.5582, -1.1725]])

In [151]:
a, o = estimate(model, train_iterator)

tensor([[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27]])

In [213]:
o

tensor([[ 2,  2,  2,  ...,  2,  2,  2],
        [13, 13, 13,  ..., 13, 13, 13],
        [13, 13, 13,  ..., 13, 13, 13],
        ...,
        [ 5,  5,  5,  ...,  5,  5,  5],
        [ 7,  7,  7,  ...,  7,  7,  7],
        [ 7,  7,  7,  ...,  7,  7,  7]])

In [212]:
o = estimate(model, train_iterator)

In [220]:
next(train_iterator.__iter__()).trg

tensor([[  2,   2,   2,  ...,   2,   2,   2],
        [  0, 174,  15,  ..., 306,  17,  15],
        [172,  50,   5,  ...,   7,   9,   5],
        ...,
        [  1,   3,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]])

In [109]:
o.shape[0] == 18

True

tensor([[[-1.1594, -2.1974, -2.0302,  ..., -1.4385, -1.2097, -1.6356],
         [-1.1594, -2.1973, -2.0301,  ..., -1.4385, -1.2096, -1.6357],
         [-1.1595, -2.1972, -2.0302,  ..., -1.4383, -1.2096, -1.6357],
         ...,
         [-1.1601, -2.1969, -2.0299,  ..., -1.4383, -1.2094, -1.6356],
         [-1.1593, -2.1974, -2.0301,  ..., -1.4384, -1.2096, -1.6357],
         [-1.1592, -2.1974, -2.0302,  ..., -1.4384, -1.2095, -1.6357]]])

In [54]:
e = next(iter(train_iterator))


In [58]:
e.src.size(-1)

128

In [52]:
next(iter(train_iterator)).trg.size()

torch.Size([26, 128])

In [22]:
class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: Tensor) -> Tuple[Tensor]:

        embedded = self.dropout(self.embedding(src))

        outputs, hidden = self.rnn(embedded)

        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        return outputs, hidden


class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(energy, dim=2)

        return F.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:

        a = self.attention(decoder_hidden, encoder_outputs)

        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        return weighted_encoder_rep


    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:

        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        return output, decoder_hidden.squeeze(0)


class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs


INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# ENC_HID_DIM = 512
# DEC_HID_DIM = 512
# ATTN_DIM = 64
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)


def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights)

optimizer = optim.Adam(model.parameters())


def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

NameError: name 'nn' is not defined

In [105]:
Tensor(data_set.train_origin.seqs.T)[:3]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [113]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, src, trg):
        self.src = Tensor(src)
        self.trg = Tensor(trg)
        
    def __getitem__(self, n):
        return self.src[n], self.trg[n]
    
    def __len__(self):
        return len(self.src)

In [114]:
data = TextDataset(data_set.train_origin.seqs.T, data_set.train_trans.seqs.T)

In [110]:
from torch.utils.data import DataLoader

In [115]:
loader = DataLoader(data, batch_size=16)

In [119]:
def train(model: nn.Module,
          loader: DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, (src, trg) in enumerate(loader):
        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(loader)


In [120]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

PAD_IDX = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')


RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.FloatTensor instead (while checking arguments for embedding)